# G-Research Crypto forecasting competition

### コンペ概要

目的：暗号通貨によるリターンの予測<br>
暗号通貨に関する時系列の価格履歴を使用して、価格が上がるか下がるか、どれだけの資産がリターンするかを予測します。<br>
本コンペでは、14種の暗号通貨についてのデータが提供されています。<br>

> 暗号通貨市場の拡大：CrypoCompare社によると、2020年における一日の平均取引量は410億ドルにも及ぶとのこと。 - [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)

データの特性の概要
- 時系列データ。
- 暗号通貨の変動は、他の暗号通貨及び暗号通貨市場全体に対して大いに影響する。
- 株価と同様に、通貨の価格はランダムウォークパターンに従っているようにも見える。

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

### データ概要確認

In [ ]:
# 学習用データの確認
data = pd.read_csv('../input/g-research-crypto-forecasting/train.csv') 
print('データ数：',data.shape[0])
data.head() 

- **timestamp**：1970-01-01 00:00:00.000 UTCからの経過秒数を示す。データは分スケールのため、60の倍数になっているようです。
- **Asset_ID**：暗号通貨を識別するID(例：ビットコインの場合はAsset_ID = 1)。 Asset_IDと暗号通貨の対応関係はasset_details.csvで参照できます。
- **Count**：時間間隔（直前）の取引の総数。
- **Open**：指定時間間隔における始値(USD)。
- **High**：指定時間間隔における最高価格(USD)。
- **Low**：指定時間間隔における最低価格(USD)。
- **Close**：指定時間間隔における終値(USD)。
- **Volume**：購入または販売された通貨の数量。
- **VWAP**：指定時間間隔における通貨の重みづけ平均価格。
- **Target**：15分間の対数収益率から算出。
timestamp、Asset_IDの複合キーが、データを識別するインデックになります。

Asset_IDと暗号通貨の対応<br>
Weightは、目的変数となっているLogReturnを算出する際に使用します。

In [ ]:
asset_tmp = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
asset = asset_tmp.set_index('Asset_ID').sort_index()
asset

### 欠損値の確認
こう見ると、ほとんどTargetの欠損だけのようです。

In [ ]:
print(data.isnull().sum())

<br>
通貨毎で欠損を確認してみます。<br>
チュートリアルによると、データ欠損は行ごと抜け落ちているパターンも存在するとのことでしたので、行ごとの欠損数も可視化します。<br>
ビットコイン、イーサリアム、ライトコインの欠損値は比較的少ないようです。

In [ ]:
missing_list = []
for asset_id, coin in zip(asset.index, asset.loc[:, 'Asset_Name']):
    asset_df = data[data["Asset_ID"]==asset_id].set_index("timestamp")
    gap_df = (asset_df.index[1:]-asset_df.index[:-1]).value_counts().reset_index()
    drop_row = int(((gap_df['index']/60-1)*gap_df['timestamp']).sum())
    missing_list.append([coin, asset_df.shape[0]] + asset_df.isna().sum().tolist() + [drop_row])

missing_df = pd.DataFrame(missing_list)
missing_df.columns = ["Asset_Name", "TotalRows", 'Missing_Asset_ID', 'Missing_Count', 'Missing_Open', 
                      'Missing_High', 'Missing_Low', 'Missing_Close', 'Missing_Volume', 'Missing_VWAP', 'Missing_Target', 'Missing_Row']
missing_df

各暗号通貨のについて提供されているデータサイズ

In [ ]:
asset_count= []
for i in range(14):
    count = (data["Asset_ID"]==i).sum()
    asset_count.append(count)
fig = px.bar(x = asset["Asset_Name"],
             y = asset_count , 
             color = asset_count ,
             color_continuous_scale="Blues") 
fig.update_xaxes(title="Assets")
fig.update_yaxes(title = "Number of Rows")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Data Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} ,
        template="plotly_white")
fig.show()

### 通貨の価格変動
株価チャートで価格変動の様子を確認してみます。plotly.graph_objectsのCandlestickで足付のチャートグラフが一発で作成できます。<br>
時間内で価格が下がっている(終値が始値よりも低い)場合はプロットが赤、反対の場合は緑になります。

In [ ]:
def crypto_df(asset_id ,data= data ):
    df = data[data["Asset_ID"]==asset_id].reset_index(drop = True)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.set_index('timestamp')
    return df

btc = crypto_df(asset_id = 1) #ビットコイン
eth = crypto_df(asset_id = 6) #イーサリアム

In [ ]:
def candelstick_chart(data,title):
    candlestick = go.Figure(data = [go.Candlestick(x =data.index, 
                                               open = data[('Open')], 
                                               high = data[('High')], 
                                               low = data[('Low')], 
                                               close = data[('Close')])])
    candlestick.update_xaxes(title_text = 'Time',
                             rangeslider_visible = True)

    candlestick.update_layout(
    title = {
        'text': '{:} Candelstick Chart'.format(title),
        'y':0.90,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'} , 
    template="plotly_white")

    candlestick.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return candlestick

ビットコイン ろうそく足チャート(ラスト50サンプル)

In [ ]:
btc_plot = candelstick_chart(btc[-50:],title = "Bitcoin(BTC)")
btc_plot.show()

OLCHチャートも簡単に書けますね。見やすい方で。

In [ ]:
def ohlc_chart(data,title):
    ohlc = go.Figure(data = [go.Ohlc(x =data.index, 
                                               open = data[('Open')], 
                                               high = data[('High')], 
                                               low = data[('Low')], 
                                               close = data[('Close')])])
    ohlc.update_xaxes(title_text = 'Time',
                             rangeslider_visible = True)

    ohlc.update_layout(
    title = {
        'text': '{:} OHLC Chart'.format(title),
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        template="plotly_white")

    ohlc.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return ohlc

ビットコイン OHLCチャート(50サンプル)

In [ ]:
ohlc_chart(btc[-50:], title = "Bitcoin(BTC)")

データを日単位でグルーピングして、時系列全体での変動を見てみます。<br>
参考：[to-the-moon-g-research-crypto-forecasting-eda](https://www.kaggle.com/iamleonie/to-the-moon-g-research-crypto-forecasting-eda)

In [ ]:
# タイムスタンプの変換
data['timestamp'] = data['timestamp'].astype('datetime64[s]')

# グルーピング
data_daily = pd.DataFrame()

for asset_id in asset.index:
    data_single = data[data.Asset_ID == asset_id].copy()

    data_single_new = data_single[['timestamp','Count']].resample('D', on='timestamp').sum()
    data_single_new['Open'] = data_single[['timestamp','Open']].resample('D', on='timestamp').first()['Open']
    data_single_new['High'] = data_single[['timestamp','High']].resample('D', on='timestamp').max()['High']
    data_single_new['Low'] = data_single[['timestamp','Low']].resample('D', on='timestamp').min()['Low']
    data_single_new['Close'] = data_single[['timestamp','Close']].resample('D', on='timestamp').last()['Close']
    data_single_new['Volume'] = data_single[['timestamp','Volume']].resample('D', on='timestamp').sum()['Volume']
    data_single_new['Asset_ID'] = asset_id

    data_daily = data_daily.append(data_single_new.reset_index(drop=False))
data_daily = data_daily.sort_values(by = ['timestamp', 'Asset_ID']).reset_index(drop=True)

data_daily = data_daily.pivot(index='timestamp', columns='Asset_ID')[['Count', 'Open', 'High', 'Low', 'Close', 'Volume']]
data_daily = data_daily.reset_index(drop=False)

In [ ]:
# ろうそく足チャート
fig = make_subplots(
    rows=len(asset.index), cols=1, subplot_titles=(asset.Asset_Name)
)

for i, asset_id in enumerate(asset.index):
    fig.append_trace(go.Candlestick(x=data_daily.timestamp, 
                                         open=data_daily[('Open', asset_id)], 
                                         high=data_daily[('High', asset_id)], 
                                         low=data_daily[('Low', asset_id)], 
                                         close=data_daily[('Close', asset_id)]),
                                         row=i+1, col=1,
                    )

    fig.update_xaxes(range=[data_daily.timestamp.iloc[0], data_daily.timestamp.iloc[-1]], row=i+1, col=1)

fig.update_layout(xaxis_rangeslider_visible = False, 
                  xaxis2_rangeslider_visible = False, 
                  xaxis3_rangeslider_visible = False,
                  xaxis4_rangeslider_visible = False,
                  xaxis5_rangeslider_visible = False,
                  xaxis6_rangeslider_visible = False,
                  xaxis7_rangeslider_visible = False,
                  xaxis8_rangeslider_visible = False,
                  xaxis9_rangeslider_visible = False,
                  xaxis10_rangeslider_visible = False,
                  xaxis11_rangeslider_visible = False,
                  xaxis12_rangeslider_visible = False,
                  xaxis13_rangeslider_visible = False,
                  xaxis14_rangeslider_visible = False,
                  height=3000, width=1000, 
                  margin = dict(l = 0, r = 0, b = 0, t = 30, pad = 0)
                 )
                 
fig.show()

### LogReturn
このコンペではTargetの算出に対数リターン(対数収益率)が使用されます。<br>
通貨によって価格スケールが異なるので、価格の変化率を計算することで通貨の価格変動を分析します。

In [ ]:
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

In [ ]:
btc_lret = log_return(btc.Close)[1000000:1500000]
eth_lret = log_return(eth.Close)[1000000:1500000]

In [ ]:
# ビットコイン、イーサネットのLogReturn
fig = go.Figure()
fig.add_trace(go.Scatter(x=btc_lret.index,
                        y=btc_lret.values,
                        #mode='line',
                        showlegend=True,
                        name='Bitcoin'))
fig.add_trace(go.Scatter(x=eth_lret.index,
                        y=eth_lret.values,
                        #mode='line',
                        showlegend=True,
                        name='Ethereum'))

fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Log Return')
fig.show()

2020年3月頃COVID-19の影響とみられる変動が確認できます。<br>
日単位のデータについてもLogReturnを可視化してみます。<br>

In [ ]:
# 全通貨LogReturn可視化
for i, asset_id in enumerate(asset.index):
    data_daily[('lret',  asset_id)] = log_return(data_daily[( 'Close',  asset_id)])

    
fig = make_subplots(
    rows=len(asset.index), cols=1, subplot_titles=(asset.Asset_Name)
)
for i, asset_id in enumerate(asset.index):
    #df_coin = data_daily[data_daily.loc[:, 'Asset_ID']==asset_id]
    fig.append_trace(go.Scatter(
                        x=data_daily.timestamp,
                        y=data_daily[('lret', asset_id)]),
                        row=i+1, col=1)

    fig.update_xaxes(range=[data_daily.timestamp.iloc[0], data_daily.timestamp.iloc[-1]], row=i+1, col=1)
                 
fig.update_layout(
                  height=3000, width=1000, 
                  margin = dict(l = 0, r = 0, b = 0, t = 30, pad = 0)
                 )
fig.show()